In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

spark = SparkSession \
    .builder \
    .master("local[3]") \
    .appName("StructuredStreamingSum") \
    .config("spark.sql.shuffle.partitions", 3) \
    .config("spark.streaming.stopGracefullyOnShutdown", "true") \
    .config("spark.sql.streaming.schemaInference", "true") \
    .config("maxFilesPerTrigger", 1) \
    .getOrCreate()


In [4]:
# brew install lucapette/tap/fakedata
# fakedata --format=ndjson --limit 10000 city domain event=event.action > streaming_sample/sample.json


df = spark \
    .readStream \
    .format("json") \
    .option("path", "test_input") \
    .load()

# df.printSchema()

In [5]:
shorten_df = df.select("city", "event")

In [ ]:
query = shorten_df \
        .writeStream \
        .format("console") \
        .option("checkpointLocation", "checkpoint") \
        .outputMode("append") \
        .trigger(processingTime='5 seconds') \
        .start()

query.awaitTermination()